# Testing GPT + Analysis

### This is just a initial project and study. I'm not a professional in this area, so I'm just trying to learn and share my knowledge. I hope you enjoy it! :)

Basically, this will work with GPT-4 + Langchain + Pinecone. Which is a new and popular stack of technologies that are being used to create a new generation of AI.

In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())

True

This cell imports the libraries and the .env keys, which are needed for this project. The keys that we are using are from OpenAI and Pinecone.

The Data that we will use comes from Amazon, so I will not upload it to GitHub. But you can download it from here: https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/ (Amazon Reviews Dataset). The ones that we are using here are the from the Amazon Fashion category.

In [2]:
data = []
with gzip.open('AMAZON_FASHION.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
        
metadata = []
with gzip.open('meta_AMAZON_FASHION.json.gz') as f:
    for l in f:
        metadata.append(json.loads(l.strip()))

Here is the explanation for the code above:

1. The first part of code is to load the Amazon Fashion dataset into a dictionary called data. 
2. The second block of code is to load the metadata for the Amazon Fashion dataset into a dictionary called metadata.

Both will be loaded in dataframes.

In [3]:
df = pd.DataFrame.from_dict(data)
df = df[df['reviewText'].notna()]

df

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2.0,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3,NaN,NaN
2,4.0,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2.0,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3.0,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
883631,5.0,True,"02 21, 2017",A1ZSB2Q144UTEY,B01HJHTH5U,Amazon Customer,I absolutely love this dress!! It's sexy and ...,I absolutely love this dress,1487635200,NaN,NaN,NaN
883632,5.0,True,"11 25, 2016",A2CCDV0J5VB6F2,B01HJHTH5U,Amazon Customer,I'm 5'6 175lbs. I'm on the tall side. I wear a...,I wear a large and ordered a large and it stil...,1480032000,2,NaN,NaN
883633,3.0,True,"11 10, 2016",A3O90PACS7B61K,B01HJHTH5U,Fabfifty,Too big in the chest area!,Three Stars,1478736000,NaN,NaN,NaN
883634,3.0,True,"11 10, 2016",A2HO94I89U3LNH,B01HJHF97K,Mgomez,"Too clear in the back, needs lining",Three Stars,1478736000,NaN,NaN,NaN


Here, we insert the data from reviews into a pandas dataframe. We will later use it to create the embeddings.

In [4]:
df_meta = pd.DataFrame.from_dict(metadata)
df_meta

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186632,JT Women's Elegant Off Shoulder Chiffon Maxi L...,JT,NaN,"9,835,890inClothing,ShoesJewelry(",5 star,B01HJGXL4O,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186633,Microcosm Retro Vintage Black Crochet Lace One...,Microcosm,[Package Dimensions:\n \n7....,"11,390,771inClothing,ShoesJewelry(",5 star5 star (0%),B01HJHF97K,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186634,Lookatool Classic Plain Vintage Army Military ...,Lookatool,"[Cotton+Polyester, Imported, Item type:Basebal...","972,275inClothing,ShoesJewelry(",5 star,B01HJGJ9LS,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,$8.53,"[B00XLECZMS, B0018MQAOY, B00N833I4Q, B074DQSPP...","[B07BHQ1FXL, B00XLECZMS, B07CJWM5WY, B07CS97C1...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
186635,Edith Windsor Women's Deep V-neck Beaded Sequi...,Edith Windsor,[Product Dimensions:\n \n9....,"1,964,585inClothing,ShoesJewelry(",5 star,B01HJHTH5U,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B077ZLGMJ3],NaN,NaN,NaN,NaN


Here we are doing almost the same, but with the metadata. We will use it to create the embeddings too.

Now, we are going to truncate the reviews so that we don't process too large or unuseful data. To do this, we will limit to 400 characters on the reviewText column.

In [5]:
max_text_length = 400
def truncate_text(text):
    return text[:max_text_length]
df['truncated']=df.apply(lambda row: truncate_text(row['reviewText']), axis=1)

In [6]:
df.groupby('asin').count().sort_values('overall')

,overall,verified,reviewTime,reviewerID,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,truncated
asin,,,,,,,,,,,,
0764443682,1,1,1,1,1,1,1,1,0,0,0,1
B019YM0O28,1,1,1,1,1,1,1,1,0,0,0,1
B019YME2N0,1,1,1,1,1,1,1,1,0,0,0,1
B019YMNS08,1,1,1,1,1,1,1,1,0,0,0,1
B00S7N99AY,1,1,1,1,1,1,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
B000PHANNM,2566,2566,2566,2566,2566,2566,2563,2566,85,2563,112,2566
B00RLSCLJM,3633,3633,3633,3633,3633,3633,3632,3633,225,3538,210,3633
B00I0VHS10,3884,3884,3884,3884,3884,3884,3880,3884,128,3872,107,3884


This cell is here just to find a product to use as an example. We will first use the second one from the bottom then the last one (from the bottom).

In [9]:
df_meta[df_meta.asin=='B000KPIHQ4'].values

array([['Powerstep Pinnacle Orthotic Shoe Insoles', nan,
        list(['Shipping Information:\n                    \nView shipping rates and policies']),
        '154inClothing,Shoesamp;Jewelry(', '5 star', 'B000KPIHQ4',
        list(['https://images-na.ssl-images-amazon.com/images/I/414VFpnmvjL._US40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/51yLLxuD5%2BL._US40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/51NJmYTkeiL._US40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41VRUCCVKEL._US40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/51b-GUTXm0L._US40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41mORzqQTwL._US40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/61RHVYCqQcL._US40_.jpg']),
        list(['https://images-na.ssl-images-amazon.com/images/I/414VFpnmvjL.jpg', 'https://images-na.ssl-images-amazon.com/images/I/51yLLxuD5%2BL.jpg', 'https://images-na.ssl-images-amazon.com/images/I/51NJmYTkeiL.jpg', 'https://images-

## Now, we will create the embeddings for the reviews and metadata.

In [10]:
df = df.loc[df['asin'] == 'B000KPIHQ4'].copy()

Here, we will use HuggingFace to create the embeddings for the reviews. This is a demo on how we can do it without OpenAI. Later, we will use OpenAI to create the embeddings.

In [11]:
from langchain.embeddings import HuggingFaceHubEmbeddings

embeddings = HuggingFaceHubEmbeddings()

ValidationError: 1 validation error for HuggingFaceHubEmbeddings
__root__
  Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass  `huggingfacehub_api_token` as a named parameter. (type=value_error)

We will create a new column and insert into the dataframe. We will use the .apply() again.

df['embeddings']=df.apply(lambda row: embeddings.embed_query(row['truncated']),axis=1)